In [164]:
# Python 3.11.2
# %pip install -r requriments.txt
# %conda install --file requriments.txt

In [165]:
# Импорт библиотек для работы с данными
import pandas as pd
import numpy as np
import scipy.stats as stats

# Импорт библиотек для визуализации данных
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.express as px

# Импорт библиотек для пред обработки данных
from sklearn.preprocessing import MaxAbsScaler, RobustScaler

# Импорт библиотек для отбора признаков
from sklearn.feature_selection import RFE, RFECV, SelectFromModel, SelectFpr, chi2

# Импорт библиотек линейной регрессии
import statsmodels.api as sm
from statsmodels.regression.recursive_ls import RecursiveLS
from statsmodels.regression.linear_model import OLS, GLS, GLSAR, WLS
from sklearn.linear_model import Ridge, Lasso, ElasticNet, SGDRegressor
from statsmodels.discrete.discrete_model import Poisson

# Импорт библиотек для машинного обучения
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet, SGDRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score

# Импорт библиотек для кросс валидации
from sklearn.model_selection import train_test_split

# Подготовка данных

## Загрузка данных

In [166]:
# Загрузка данных с помощью pandas из файла csv
data = pd.read_csv("input/data.csv", delimiter=";")

# Просмотр первых 5 строк данных
data.head()

,Год,y1,y2,y3,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x11
0,1998,87211.83,328497.9,638450.60,228548.4,803.2,5780.87,1212184.0,33.8,83359.41,25719.20,6171.0,6642.0,111458.0,1393.30
1,1999,119302.33,695059.8,789466.92,488395.1,787.6,5823.01,1272744.0,35.2,87820.24,25262.15,7260.0,7814.0,185861.0,1807.80
2,2000,156215.00,1159034.0,962057.00,748241.8,1472.8,5865.15,1444737.0,32.4,92700.80,25720.20,8067.0,9194.0,309534.0,2185.00
3,2001,173839.00,1370182.8,1393532.20,1008088.5,1154.6,5907.28,1841258.0,30.4,98088.36,24905.88,5545.0,12637.0,418289.0,2385.26
4,2002,220396.00,1767476.7,1771073.00,1267935.2,1508.7,5961.24,2255912.0,35.1,104100.59,25084.01,6932.0,13817.0,589139.0,2918.55


In [167]:
# Удаление столбца с годом так как он не несет полезной информации для составления модели
year = data['Год']
data = data.drop(columns=['Год'])
data.head()

,y1,y2,y3,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x11
0,87211.83,328497.9,638450.60,228548.4,803.2,5780.87,1212184.0,33.8,83359.41,25719.20,6171.0,6642.0,111458.0,1393.30
1,119302.33,695059.8,789466.92,488395.1,787.6,5823.01,1272744.0,35.2,87820.24,25262.15,7260.0,7814.0,185861.0,1807.80
2,156215.00,1159034.0,962057.00,748241.8,1472.8,5865.15,1444737.0,32.4,92700.80,25720.20,8067.0,9194.0,309534.0,2185.00
3,173839.00,1370182.8,1393532.20,1008088.5,1154.6,5907.28,1841258.0,30.4,98088.36,24905.88,5545.0,12637.0,418289.0,2385.26
4,220396.00,1767476.7,1771073.00,1267935.2,1508.7,5961.24,2255912.0,35.1,104100.59,25084.01,6932.0,13817.0,589139.0,2918.55


### Объяснение переменных
- $y_1$ - инвестиции в основной капитал, млн руб.
- $y_2$ - валовой региональный продукт (ВРП), млн.руб.
- $y_3$ - сумма доходов населения за год, млн руб.
- $x_1$ - финансовый результат деятельности (чистая прибыль)
- $x_2$ - прямые иностранные инвестиции, млн USD
- $x_3$ - среднегодовая численности занятых, тыс чел.
- $x_4$ - стоимость основных фондов, млн. руб
- $x_5$ - степень износа основных фондов, %
- $x_6$ - затраты на научные исследования и разработки, млн руб.
- $x_7$ - объём инновационных товаров работ услуг, млн руб.
- $x_8$ - экспорт, млн USD
- $x_9$ - импорт, млн. USD
- $x_{10}$ - сумма остатков вкладов на счетах в Банке России, млн. руб.
- $x_{11}$ - прожиточный минимум в регионе РФ (г. Москва), тыс.руб.

## Изучение данных на наличие ошибок измерений

Функция combine_scatter_plots строит графики рассеяния для каждой пары признаков в наборе данных.

In [168]:
def combine_scatter_plots(height=1000, width=1100):
    fig = make_subplots(rows=5, cols=3)
    for i, column in enumerate(data.columns.values):
        if 0 <= i < 3:
            row, col = 1, i + 1
        elif 3 <= i < 6:
            row, col = 2, i + 1 - 3
        elif 6 <= i < 9:
            row, col = 3, i + 1 - 6
        elif 9 <= i < 12:
            row, col = 4, i + 1 - 9
        else:
            row, col = 5, i + 1 - 12

        if 0 <= i < 3:
            fig.add_trace(go.Scatter(y=data[f'y{i + 1}'], x=year, name=column), row=row, col=col)
        else:
            fig.add_trace(go.Scatter(y=data[f'x{i - 2}'], x=year, name=column), row=row, col=col)
    fig.update_layout(height=height, width=width, title_text="Зависимость признаков от времени")
    fig.show()

In [169]:
combine_scatter_plots()

## Проверка данных на наличие выбросов и изучение распределения данных

Функция combine_box_plots строит ящиковые диаграммы для каждого признака в наборе данных, где значения признака разбиваются на квартили и представляются в виде ящика.

Функция combine_violin_plots строит аналогичные диаграммы, но вместо ящика используется график, который представляет распределение значений признака с помощью ядерной оценки плотности.

In [170]:
# Построение графиков
def combine_box_plots(height=1000, width=1100):
    fig = make_subplots(rows=5, cols=3)
    for i, column in enumerate(data.columns.values):
        if 0 <= i < 3:
            row, col = 1, i + 1
        elif 3 <= i < 6:
            row, col = 2, i + 1 - 3
        elif 6 <= i < 9:
            row, col = 3, i + 1 - 6
        elif 9 <= i < 12:
            row, col = 4, i + 1 - 9
        else:
            row, col = 5, i + 1 - 12

        if 0 <= i < 3:
            fig.add_trace(go.Box(x=data[f'y{i + 1}'], name=column), row=row, col=col)
        else:
            fig.add_trace(go.Box(x=data[f'x{i - 2}'], name=column), row=row, col=col)
    fig.update_layout(height=height, width=width, title_text="Ящиковые диаграммы распределения признаков",
                      showlegend=False)
    fig.show()


def combine_violin_plots(height=1500, width=1100):
    fig = make_subplots(rows=5, cols=3)
    for i, column in enumerate(data.columns.values):
        if 0 <= i < 3:
            row, col = 1, i + 1
        elif 3 <= i < 6:
            row, col = 2, i + 1 - 3
        elif 6 <= i < 9:
            row, col = 3, i + 1 - 6
        elif 9 <= i < 12:
            row, col = 4, i + 1 - 9
        else:
            row, col = 5, i + 1 - 12

        if 0 <= i < 3:
            fig.add_trace(go.Violin(x=data[f'y{i + 1}'], name=column), row=row, col=col)
        else:
            fig.add_trace(go.Violin(x=data[f'x{i - 2}'], name=column), row=row, col=col)
    fig.update_layout(height=height, width=width, title_text="Распределение признаков", showlegend=False)
    fig.show()

In [171]:
combine_box_plots()

In [172]:
combine_violin_plots()

## Избавление от мультиколлинеарности между признаками путем нормализации и центрирования данных

In [173]:
# Центрирование данных для избавления от мультиколлинеарности
# data = data.apply(lambda x: x - x.mean())

# Минимаксная нормализация данных
data = data.apply(lambda x: (x - x.min()) / (x.max() - x.min()))

# Нормализация средним (Z-нормализация)
# data = data.apply(lambda x: (x - x.mean()) / x.std())

# MaxAbsScaler
# transformer = MaxAbsScaler().fit(data)
# data = pd.DataFrame(transformer.transform(data))

# RobustScaler
# transformer = RobustScaler().fit(data)
# data = pd.DataFrame(transformer.transform(data))

data.columns = ['y1', 'y2', 'y3', 'x1', 'x2', 'x3', 'x4', 'x5', 'x6', 'x7', 'x8', 'x9', 'x10', 'x11']
data.head()

,y1,y2,y3,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x11
0,0.000000,0.000000,0.000000,0.000000,0.000098,0.000000,0.000000,0.201183,0.000000,0.001083,0.002159,0.000000,0.000000,0.000000
1,0.006149,0.016090,0.011180,0.028630,0.000000,0.050591,0.000802,0.284024,0.010861,0.000596,0.005915,0.009124,0.001756,0.023931
2,0.013222,0.036457,0.023958,0.057260,0.004307,0.101183,0.003081,0.118343,0.022745,0.001084,0.008698,0.019866,0.004676,0.045708
3,0.016599,0.045725,0.055902,0.085890,0.002307,0.151762,0.008333,0.000000,0.035863,0.000217,0.000000,0.046669,0.007243,0.057270
4,0.025519,0.063165,0.083854,0.114519,0.004533,0.216544,0.013826,0.278107,0.050502,0.000406,0.004784,0.055855,0.011277,0.088059


# Матрица коэффициентов межфакторной корреляции

In [174]:
un_data = data.copy()

fig = px.imshow(un_data.corr(), text_auto='.2f', color_continuous_scale="gnbu")
# fig = px.imshow(un_data.corr(), text_auto='.2f', color_continuous_scale="matter")
fig.update_layout(height=1000, title_text="Матрица коэффициентов межфакторной корреляции")
fig.show()

In [175]:
# un_data.columns = ['Инвестиции в основной капитал, млн руб. y1', 'Валовой региональный продукт (ВРП), млн.руб. y2',
#                    'Сумма доходов населения за год, млн руб. y3',
#                    'Финансовый результат деятельности (чистая прибыль), млн.руб. x1',
#                    'Прямые иностранные инвестиции, млн USD x2',
#                    'Среднегодовая численности занятых, тыс чел. x3', 'Стоимость основных фондов, млн. руб. x4',
#                    'Степень износа основных фондов, % x5',
#                    'Затраты на научные исследования и разработки, млн руб. x6',
#                    'Объём инновационных товаров работ услуг, млн руб. x7', 'Экспорт, млн USD x8',
#                    'Импорт, млн. USD x9', 'Сумма остатков вкладов на счетах в Банке России, млн. руб. x10',
#                    'Прожиточный минимум в регионе РФ (г. Москва), тыс.руб. x11']
#
# fig, ax = plt.subplots(figsize=(10, 8))
# # sns.heatmap(data.corr(), annot=True, fmt=".2f", linewidth=.5, cmap='magma')
# sns.heatmap(un_data.corr(), annot=True, fmt=".2f", linewidth=.5, cmap='cubehelix')

# Исследование данных на наличие выбросов

Нахождение выбросов - это один из методов анализа данных, который используется для выявления аномальных значений в наборе данных. Выбросы могут возникать из-за ошибок в данных, случайных вариаций или из-за наличия редких, но реальных экстремальных значений.

Существует несколько методов нахождения выбросов, в том числе метод Z-оценки и метод межквартильного размаха (IQR).

In [176]:
# Смотрим размерность данных
data.shape

(25, 14)

Изначально мы имели набор данных, состоящий из 25 значений, охватывающих период с 1998 по 2022 год. На это нам указывает размерность данных (25, 14).

**Z-оценка**

Метод Z-оценки основан на расчете Z-оценки для каждого значения в наборе данных и проверке, насколько далеко каждое значение от среднего значения. Значения, которые находятся на расстоянии больше чем три стандартных отклонения от среднего, могут быть считаны выбросами.

In [177]:
z = np.abs(stats.zscore(data))
data_clean = data[(z < 3).all(axis=1)]
data_clean.shape

(25, 14)

**Межквартильный размах (IQR)**

Метод межквартильного размаха (IQR) основан на вычислении интерквартильного размаха - разницы между 75-перцентилью и 25-перцентилем. Затем определяется верхняя и нижняя границы выбросов, которые определяются как 1,5 межквартильных размаха за пределами верхнего и нижнего квартилей.

In [178]:
# Находим Первый квартиль (Q1) и Третий квартиль (Q3) и рассчитываем межквартильный размах
Q1 = data.quantile(q=.25)
Q3 = data.quantile(q=.75)
IQR = data.apply(stats.iqr)

# Оставляем только те значения, которые больше нижней границы и меньше верхней границы.
data_clean = data[~((data < (Q1 - 1.5 * IQR)) | (data > (Q3 + 1.5 * IQR))).any(axis=1)]
data_clean.shape  # без 20,21,22 годов

(25, 14)

В результате проверки на наличие выбросов мы получили исходный набор данных, состоящий из 25 значений, охватывающих период с 1998 по 2022 год. На это нам указывает размерность данных (25, 14). Это говорит нам о том что выбросов в данных найденно не было.

In [179]:
# Заменяем исходный набор данных на очищенный
data = data_clean

# Построение модели линейной регрессии

Линейная регрессия - это метод анализа данных, который используется для описания и прогнозирования линейной связи между зависимой и одной или несколькими независимыми переменными. Он часто используется в статистическом анализе данных для построения математической модели, которая может быть использована для прогнозирования значений зависимой переменной на основе значений независимых переменных.

Для построения модели линейной регрессии необходимо иметь набор данных, который состоит из пар значений зависимой и независимой переменных. Затем на основе этих данных проводится анализ, который позволяет оценить параметры модели - коэффициенты наклона и пересечения. Эти параметры могут быть использованы для построения уравнения линейной регрессии.

Для оценки качества модели линейной регрессии используются различные статистические показатели, такие как коэффициент детерминации (R-квадрат) или корреляционный коэффициент. Они позволяют оценить, насколько хорошо модель соответствует данным и насколько точно она может использоваться для прогнозирования значений зависимой переменной.

Важно понимать, что линейная регрессия является моделью, которая описывает только линейные отношения между переменными. Если связь между переменными не является линейной, то модель линейной регрессии может оказаться неэффективной и требовать использования других методов анализа данных.

In [180]:
class CustomLinearRegression:
    def __init__(self, y, x, model_type=OLS):
        self.model_type = model_type
        self.y = y
        self.x = x
        self.model = self.model_type(self.y, self.x).fit()

    def get_coefficients(self):
        print(self.model.params)
        answer = []
        names = self.x.columns.values.tolist()
        for i, par in enumerate(self.model.params):
            answer.append(f'+ {par:.4f}{names[i][0]}_{{{names[i][1:]}}}')
        print(f'$$\hat {self.y.name[0]}_{self.y.name[1]} = {" ".join(answer)[2:]}$$')

    def summary(self):
        print(self.model.summary())

    def pre(self):
        return self.model.predict(self.x)

    def F(self):
        return self.model.fvalue

    def current_p(self):
        return self.model.f_pvalue

    def forward_selection(self):
        xi = []
        stop = False
        x_new = []  # список переменных, которые будут включены в модель
        x_len = len(self.x.columns) - 1  # количество столбцов в датафрейме
        x_test = []
        for n in range(x_len):
            F_max = 0  # максимальное значение F
            p_max = 0  # максимальное значение p
            F_max_i = 0
            F_max_x = ''
            if not stop:
                for i in range(x_len):
                    if i not in xi:
                        x_test = x_new.copy()
                        x_test.append(self.x.columns[i])
                        F = CustomLinearRegression(self.y, self.x.loc[:, x_test]).F()
                        print(f"'x{i + 1}',", F)
                        if F > F_max:
                            F_max = F
                            F_max_i = i
                            F_max_x = f"x{i + 1}"
                x_new.append(F_max_x)
                xi.append(F_max_i)
                print(x_new)
                for j in range(len(x_new)):
                    p = CustomLinearRegression(self.y, self.x.loc[:, x_new]).current_p(j)
                    if p > 0.05:
                        stop = True
                        print(f'P-value: {p}')
                        break
        CustomLinearRegression(self.y, self.x.loc[:, x_new[:-1]]).summary()

    def backward_elimination(self, p=0.05, start=0):
        regression = self.model_type(self.y, self.x).fit()
        futures_num = len(self.x.columns)
        for i in range(futures_num):
            # self.x = sm.add_constant(self.x)
            regression = self.model_type(self.y, self.x).fit()
            max_p = max(regression.pvalues.values[start:])
            if max_p > p:
                for j in range(0, futures_num - i):
                    if regression.pvalues[j] == max_p:
                        self.x = self.x.drop(self.x.columns[[j]], axis=1)
        print(regression.summary())

    def custom_feature_selection(x, y, model=LinearRegression(), selection_method=RFE):
        result_list = list()
        result = selection_method(model).fit(x, y)
        for i, feature in enumerate(result.get_support()):
            if feature:
                result_list.append(f'x{i + 1}')
        CustomLinearRegression(y, x[result_list]).summary()

    def correlation_map(self):
        return px.imshow(self.x.corr(), text_auto='.2f', color_continuous_scale="gnbu")


class CustomLinearRegressionWithConst(CustomLinearRegression):
    def __init__(self, y, x, model_type=OLS):
        super().__init__(y, x, model_type)
        self.x = sm.add_constant(self.x)
        self.model = self.model_type(self.y, self.x).fit()


class CustomLinearRegressionReg(CustomLinearRegression):
    def __init__(self, y, x, model_type=OLS):
        super().__init__(y, x, model_type)
        self.model = sm.OLS(self.y, self.x).fit_regularized(alpha=1., L1_wt=0.5, refit=True)

# class CustomRidgeLinearRegression(CustomLinearRegression):
#     def __init__(self, y, x, model_type=OLS):
#         super().__init__(y, x, model_type)
#         self.model = Ridge(alpha=0.1).fit(self.x, self.y)

# Отбор переменных для модели линейной регрессии

Отбор переменных для модели линейной регрессии - это процесс выбора наиболее важных и значимых независимых переменных, которые будут использоваться в модели для прогнозирования зависимой переменной. Это важный шаг в построении модели, так как использование неподходящих переменных может привести к низкой точности прогнозов и низкому качеству модели.

Существует несколько методов отбора переменных для модели линейной регрессии, включая:

- Метод пошагового отбора переменных - это метод, который включает или исключает независимые переменные в модель пошагово на основе их значимости. На каждом шаге происходит выбор переменной, которая наиболее улучшает качество модели.

- Метод отбора на основе значимости - этот метод основан на анализе значимости каждой независимой переменной с помощью статистических тестов, таких как t-тест или F-тест. Переменные с низким уровнем значимости исключаются из модели.

- Метод регуляризации - это метод, который добавляет штрафы за большие значения коэффициентов в модель. Он позволяет автоматически отбирать переменные, которые наиболее важны для модели, и уменьшает вероятность переобучения.

- Метод отбора на основе информационных критериев - этот метод основан на использовании различных информационных критериев, таких как AIC (критерий Акаике) и BIC (критерий Шварца), чтобы выбрать модель с наименьшей ошибкой.

Выбор метода отбора переменных зависит от конкретной задачи и объема доступных данных. Важно также учитывать возможную мультиколлинеарность между независимыми переменными, что может привести к искажению результатов. Поэтому важно проводить анализ и выбор переменных внимательно и осознанно.

Сначала посмотрим на данные, которые мы будем использовать для построения модели линейной регрессии.

In [181]:
data.head()

,y1,y2,y3,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x11
0,0.000000,0.000000,0.000000,0.000000,0.000098,0.000000,0.000000,0.201183,0.000000,0.001083,0.002159,0.000000,0.000000,0.000000
1,0.006149,0.016090,0.011180,0.028630,0.000000,0.050591,0.000802,0.284024,0.010861,0.000596,0.005915,0.009124,0.001756,0.023931
2,0.013222,0.036457,0.023958,0.057260,0.004307,0.101183,0.003081,0.118343,0.022745,0.001084,0.008698,0.019866,0.004676,0.045708
3,0.016599,0.045725,0.055902,0.085890,0.002307,0.151762,0.008333,0.000000,0.035863,0.000217,0.000000,0.046669,0.007243,0.057270
4,0.025519,0.063165,0.083854,0.114519,0.004533,0.216544,0.013826,0.278107,0.050502,0.000406,0.004784,0.055855,0.011277,0.088059


Мы имеем дело с набором данных, содержащим 11 независимых переменных и 3 зависимых переменных. Переменные x1-x11 являются независимыми переменными, а переменные y1-y3 являются зависимыми переменными. Всего у нас 25 наблюдений. Разделим данные на независимые и зависимые переменные.

### Разделение данных на зависимые и независимые переменные

Зависимые переменные - это переменные, значения которых мы хотим предсказать. В нашем случае это переменные y1, y2 и y3.

In [182]:
y = data[['y1', 'y2', 'y3']]
y1 = y['y1']
y2 = y['y2']
y3 = y['y3']

Независимые переменные - это переменные, значения которых мы используем для прогнозирования зависимых переменных. В нашем случае это переменные x1-x11.

In [183]:
X = data[['x1', 'x2', 'x3', 'x4', 'x5', 'x6', 'x7', 'x8', 'x9', 'x10', 'x11']]
x1 = X['x1']
x2 = X['x2']
x3 = X['x3']
x4 = X['x4']
x5 = X['x5']
x6 = X['x6']
x7 = X['x7']
x8 = X['x8']
x9 = X['x9']
x10 = X['x10']
x11 = X['x11']

## Оценка предварительно составленных систем уравнений

Всего у нас три системы уравнений, которые мы можем использовать для прогнозирования зависимых переменных
- Система независимых уравнений;
- Система рекурсивных уравнений;
- Система одновременных уравнений.

#### Система независимых уравнений
$$
\begin{cases}
y_1 = a_1 + b_{11}x_{1} + b_{12}x_{2} + \epsilon \\
y_2 = a_2 + b_{21}x_{1} + b_{22}x_{2} + b_{23}x_{3} + b_{24}x_{4} + b_{25}x_{5} + b_{26}x_{6} + b_{27}x_{7} + b_{28}x_{8} + b_{29}x_{9} + \epsilon \\
y_3 = a_3 + b_{31}x_{1} + b_{32}x_{2} + b_{33}x_{3} + b_{34}x_{4} + b_{35}x_{5} + b_{36}x_{6} + b_{37}x_{7} + b_{38}x_{8} + b_{39}x_{9} + b_{310}x_{10} + b_{311}x_{11} + \epsilon \\
\end{cases}
$$

#### Система рекурсивных уравнений
$$
\begin{cases}
y_1 = a_1 + b_{11}x_{1} + b_{12}x_{2} + \epsilon \\
y_2 = a_2 + a_{21}y_1 + b_{21}x_{3} + b_{22}x_{4} + b_{23}x_{5} + b_{24}x_{6} + b_{25}x_{7} + b_{26}x_{8} + b_{27}x_{9} + \epsilon \\
y_3 = a_3 + a_{31}y_1 + a_{32}y_2 + b_{31}x_{11} + \epsilon \\
\end{cases}
$$

#### Система одновременных уравнений
$$
\begin{cases}
y_1 = a_1 + b_{11}x_{1} + b_{12}x_{2} + \epsilon \\
y_2 = a_2 + a_{21}y_1 + b_{23}x_{3} + b_{24}x_{4} + b_{25}x_{5} + b_{26}x_{6} + b_{27}x_{7} + b_{28}x_{8} + b_{29}x_{9} + \epsilon \\
y_3 = a_3 + a_{31}y_1 + b_{310}x_{10} + b_{311}x_{11} + \epsilon \\
\end{cases}
$$

## Система независимых уравнений

$$
\begin{cases}
y_1 = a_1 + b_{11}x_{1} + b_{12}x_{2} + \epsilon \\
y_2 = a_2 + b_{21}x_{1} + b_{22}x_{2} + b_{23}x_{3} + b_{24}x_{4} + b_{25}x_{5} + b_{26}x_{6} + b_{27}x_{7} + b_{28}x_{8} + b_{29}x_{9} + \epsilon \\
y_3 = a_3 + b_{31}x_{1} + b_{32}x_{2} + b_{33}x_{3} + b_{34}x_{4} + b_{35}x_{5} + b_{36}x_{6} + b_{37}x_{7} + b_{38}x_{8} + b_{39}x_{9} + b_{310}x_{10} + b_{311}x_{11} + \epsilon \\
\end{cases}
$$

### Первое уравнение

$$y_1 = a_1 + b_{11}x_{1} + b_{12}x_{2} + \epsilon $$

#### Исходная система уравнений

In [184]:
fn1_nez = pd.concat([x1, x2], axis=1)
CustomLinearRegression(y1, fn1_nez).summary()

                                 OLS Regression Results                                
Dep. Variable:                     y1   R-squared (uncentered):                   0.873
Model:                            OLS   Adj. R-squared (uncentered):              0.862
Method:                 Least Squares   F-statistic:                              78.77
Date:                Mon, 24 Apr 2023   Prob (F-statistic):                    5.12e-11
Time:                        21:03:16   Log-Likelihood:                          13.179
No. Observations:                  25   AIC:                                     -22.36
Df Residuals:                      23   BIC:                                     -19.92
Df Model:                           2                                                  
Covariance Type:            nonrobust                                                  
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------

#### Исходная система уравнений после отбора признаков методом обратного исключения (Backward elimination)

In [185]:
CustomLinearRegression(y1, fn1_nez).backward_elimination()

                                 OLS Regression Results                                
Dep. Variable:                     y1   R-squared (uncentered):                   0.873
Model:                            OLS   Adj. R-squared (uncentered):              0.862
Method:                 Least Squares   F-statistic:                              78.77
Date:                Mon, 24 Apr 2023   Prob (F-statistic):                    5.12e-11
Time:                        21:03:16   Log-Likelihood:                          13.179
No. Observations:                  25   AIC:                                     -22.36
Df Residuals:                      23   BIC:                                     -19.92
Df Model:                           2                                                  
Covariance Type:            nonrobust                                                  
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------

#### Конечная система уравнений и веса

In [186]:
CustomLinearRegression(y1, pd.concat([x1, x2], axis=1)).get_coefficients()

x1    0.373605
x2    0.315693
dtype: float64
$$\hat y_1 = 0.3736x_{1} + 0.3157x_{2}$$


$$\hat y_1 = 0.3736x_{1} + 0.3157x_{2}$$

In [187]:
fig = make_subplots(rows=1, cols=2)
fig.update_yaxes(title_text='Y1', row=1, col=1)

fig.add_trace(px.scatter(x=x1, y=y1).data[0], row=1, col=1)
fig.update_xaxes(title_text='X1', row=1, col=1)

fig.add_trace(px.scatter(x=x2, y=y1).data[0], row=1, col=2)
fig.update_xaxes(title_text='X2', row=1, col=2)

fig.update_layout(title_text="График рассеяния")
fig.show()

### Второе уравнение

$$y_2 = a_2 + b_{21}x_{1} + b_{22}x_{2} + b_{23}x_{3} + b_{24}x_{4} + b_{25}x_{5} + b_{26}x_{6} + b_{27}x_{7} + b_{28}x_{8} + b_{29}x_{9} + \epsilon$$

#### Исходная система уравнений

In [188]:
fn2_nez = pd.concat([x1, x2, x3, x4, x5, x6, x7, x8, x9], axis=1)
CustomLinearRegression(y2, fn2_nez).summary()

                                 OLS Regression Results                                
Dep. Variable:                     y2   R-squared (uncentered):                   0.997
Model:                            OLS   Adj. R-squared (uncentered):              0.996
Method:                 Least Squares   F-statistic:                              684.7
Date:                Mon, 24 Apr 2023   Prob (F-statistic):                    5.86e-19
Time:                        21:03:16   Log-Likelihood:                          55.388
No. Observations:                  25   AIC:                                     -92.78
Df Residuals:                      16   BIC:                                     -81.81
Df Model:                           9                                                  
Covariance Type:            nonrobust                                                  
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------

#### Исходная система уравнений после отбора признаков методом обратного исключения (Backward elimination)

In [189]:
CustomLinearRegression(y2, fn2_nez).backward_elimination()

                                 OLS Regression Results                                
Dep. Variable:                     y2   R-squared (uncentered):                   0.997
Model:                            OLS   Adj. R-squared (uncentered):              0.996
Method:                 Least Squares   F-statistic:                              2125.
Date:                Mon, 24 Apr 2023   Prob (F-statistic):                    3.07e-27
Time:                        21:03:16   Log-Likelihood:                          51.845
No. Observations:                  25   AIC:                                     -97.69
Df Residuals:                      22   BIC:                                     -94.03
Df Model:                           3                                                  
Covariance Type:            nonrobust                                                  
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------

#### Конечная система уравнений и веса

In [216]:
CustomLinearRegression(y2, pd.concat([x1, x6, x7], axis=1)).get_coefficients()

x1    0.193072
x6    0.891791
x7   -0.080006
dtype: float64
$$\hat y_2 = 0.1931x_{1} + 0.8918x_{6} + -0.0800x_{7}$$


$$\hat y_2 = 0.1931x_{1} + 0.8918x_{6} - 0.0800x_{7}$$

In [217]:
fig = make_subplots(rows=1, cols=3)
fig.update_yaxes(title_text='Y2', row=1, col=1)

fig.add_trace(px.scatter(x=x1, y=y2).data[0], row=1, col=1)
fig.update_xaxes(title_text='X1', row=1, col=1)

fig.add_trace(px.scatter(x=x6, y=y2).data[0], row=1, col=2)
fig.update_xaxes(title_text='X6', row=1, col=2)

fig.add_trace(px.scatter(x=x7, y=y2).data[0], row=1, col=3)
fig.update_xaxes(title_text='X7', row=1, col=3)

fig.update_layout(title='Графики с линиями тренда')
fig.show()

### Третье уравнение

$$y_3 = a_3 + b_{31}x_{1} + b_{32}x_{2} + b_{33}x_{3} + b_{34}x_{4} + b_{35}x_{5} + b_{36}x_{6} + b_{37}x_{7} + b_{38}x_{8} + b_{39}x_{9} + b_{310}x_{10} + b_{311}x_{11} + \epsilon$$

#### Исходная система уравнений

In [218]:
fn3_nez = pd.concat([x1, x2, x3, x4, x5, x6, x7, x8, x9, x10, x11], axis=1)
CustomLinearRegression(y3, fn3_nez).summary()

                                 OLS Regression Results                                
Dep. Variable:                     y3   R-squared (uncentered):                   0.999
Model:                            OLS   Adj. R-squared (uncentered):              0.998
Method:                 Least Squares   F-statistic:                              1352.
Date:                Mon, 24 Apr 2023   Prob (F-statistic):                    3.36e-19
Time:                        21:07:21   Log-Likelihood:                          68.297
No. Observations:                  25   AIC:                                     -114.6
Df Residuals:                      14   BIC:                                     -101.2
Df Model:                          11                                                  
Covariance Type:            nonrobust                                                  
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------

#### Исходная система уравнений после отбора признаков методом обратного исключения (Backward elimination)

In [219]:
CustomLinearRegression(y3, fn3_nez).backward_elimination()

                                 OLS Regression Results                                
Dep. Variable:                     y3   R-squared (uncentered):                   0.999
Model:                            OLS   Adj. R-squared (uncentered):              0.999
Method:                 Least Squares   F-statistic:                              3455.
Date:                Mon, 24 Apr 2023   Prob (F-statistic):                    1.21e-28
Time:                        21:07:22   Log-Likelihood:                          65.713
No. Observations:                  25   AIC:                                     -121.4
Df Residuals:                      20   BIC:                                     -115.3
Df Model:                           5                                                  
Covariance Type:            nonrobust                                                  
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------

#### Конечная система уравнений и веса

In [194]:
CustomLinearRegression(y3, pd.concat([x1, x3, x4, x5, x10], axis=1)).get_coefficients()

x1     0.191874
x3     0.132485
x4     0.284618
x5     0.055689
x10    0.361107
dtype: float64
$$\hat y_3 = 0.1919x_{1} + 0.1325x_{3} + 0.2846x_{4} + 0.0557x_{5} + 0.3611x_{10}$$


$$\hat y_3 = 0.1919x_{1} + 0.1325x_{3} + 0.2846x_{4} + 0.0557x_{5} + 0.3611x_{10}$$

In [220]:
fig = make_subplots(rows=2, cols=3)
fig.update_yaxes(title_text='Y3', row=1, col=1)
fig.update_yaxes(title_text='Y3', row=2, col=1)

fig.add_trace(px.scatter(x=x1, y=y3).data[0], row=1, col=1)
fig.update_xaxes(title_text='X1', row=1, col=1)

fig.add_trace(px.scatter(x=x3, y=y3).data[0], row=1, col=2)
fig.update_xaxes(title_text='X3', row=1, col=2)

fig.add_trace(px.scatter(x=x4, y=y3).data[0], row=1, col=3)
fig.update_xaxes(title_text='X4', row=1, col=3)

fig.add_trace(px.scatter(x=x5, y=y3).data[0], row=2, col=1)
fig.update_xaxes(title_text='X5', row=2, col=1)

fig.add_trace(px.scatter(x=x10, y=y3).data[0], row=2, col=2)
fig.update_xaxes(title_text='X10', row=2, col=2)

fig.update_layout(height=800, title='Графики с линиями тренда')
fig.show()

## Система рекурсивных уравнений

$$
\begin{cases}
y_1 = a_1 + b_{11}x_{1} + b_{12}x_{2} + \epsilon \\
y_2 = a_2 + a_{21}y_1 + b_{21}x_{3} + b_{22}x_{4} + b_{23}x_{5} + b_{24}x_{6} + b_{25}x_{7} + b_{26}x_{8} + b_{27}x_{9} + \epsilon \\
y_3 = a_3 + a_{31}y_1 + a_{32}y_2 + b_{31}x_{11} + \epsilon \\
\end{cases}
$$

### Первое уравнение

$$y_1 = a_1 + b_{11}x_{1} + b_{12}x_{2} + \epsilon$$

#### Исходная система уравнений

In [221]:
fn1_rec = pd.concat([x1, x2], axis=1)
CustomLinearRegression(y1, fn1_rec).summary()

                                 OLS Regression Results                                
Dep. Variable:                     y1   R-squared (uncentered):                   0.873
Model:                            OLS   Adj. R-squared (uncentered):              0.862
Method:                 Least Squares   F-statistic:                              78.77
Date:                Mon, 24 Apr 2023   Prob (F-statistic):                    5.12e-11
Time:                        21:07:55   Log-Likelihood:                          13.179
No. Observations:                  25   AIC:                                     -22.36
Df Residuals:                      23   BIC:                                     -19.92
Df Model:                           2                                                  
Covariance Type:            nonrobust                                                  
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------

#### Исходная система уравнений после отбора признаков методом обратного исключения (Backward elimination)

In [222]:
CustomLinearRegression(y1, fn1_rec).backward_elimination()

                                 OLS Regression Results                                
Dep. Variable:                     y1   R-squared (uncentered):                   0.873
Model:                            OLS   Adj. R-squared (uncentered):              0.862
Method:                 Least Squares   F-statistic:                              78.77
Date:                Mon, 24 Apr 2023   Prob (F-statistic):                    5.12e-11
Time:                        21:08:02   Log-Likelihood:                          13.179
No. Observations:                  25   AIC:                                     -22.36
Df Residuals:                      23   BIC:                                     -19.92
Df Model:                           2                                                  
Covariance Type:            nonrobust                                                  
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------

#### Конечная система уравнений и веса

In [223]:
CustomLinearRegression(y1, pd.concat([x1, x2], axis=1)).get_coefficients()

x1    0.373605
x2    0.315693
dtype: float64
$$\hat y_1 = 0.3736x_{1} + 0.3157x_{2}$$


$$\hat y_1 = 0.3736x_{1} + 0.3157x_{2}$$

### Второе уравнение

$$y_2 = a_2 + a_{21}y_1 + b_{21}x_{3} + b_{22}x_{4} + b_{23}x_{5} + b_{24}x_{6} + b_{25}x_{7} + b_{26}x_{8} + b_{27}x_{9} + \epsilon$$

#### Исходная система уравнений

In [224]:
fn2_rec = pd.concat([y1, x1, x2, x3, x4, x5, x6, x7, x8, x9], axis=1)
CustomLinearRegression(y2, fn2_rec).summary()

                                 OLS Regression Results                                
Dep. Variable:                     y2   R-squared (uncentered):                   0.997
Model:                            OLS   Adj. R-squared (uncentered):              0.996
Method:                 Least Squares   F-statistic:                              578.3
Date:                Mon, 24 Apr 2023   Prob (F-statistic):                    1.59e-17
Time:                        21:08:17   Log-Likelihood:                          55.400
No. Observations:                  25   AIC:                                     -90.80
Df Residuals:                      15   BIC:                                     -78.61
Df Model:                          10                                                  
Covariance Type:            nonrobust                                                  
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------

#### Исходная система уравнений после отбора признаков методом обратного исключения (Backward elimination)

In [228]:
CustomLinearRegression(y2, fn2_rec).backward_elimination(start=1)

                                 OLS Regression Results                                
Dep. Variable:                     y2   R-squared (uncentered):                   0.997
Model:                            OLS   Adj. R-squared (uncentered):              0.996
Method:                 Least Squares   F-statistic:                              1635.
Date:                Mon, 24 Apr 2023   Prob (F-statistic):                    7.34e-26
Time:                        21:09:38   Log-Likelihood:                          52.737
No. Observations:                  25   AIC:                                     -97.47
Df Residuals:                      21   BIC:                                     -92.60
Df Model:                           4                                                  
Covariance Type:            nonrobust                                                  
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------

#### Конечная система уравнений и веса

In [229]:
CustomLinearRegression(y2, pd.concat([y1, x1, x3, x6], axis=1)).get_coefficients()

y1    0.133910
x1    0.383441
x3   -0.115785
x6    0.625372
dtype: float64
$$\hat y_2 = 0.1339y_{1} + 0.3834x_{1} + -0.1158x_{3} + 0.6254x_{6}$$


$$\hat y_2 = 0.1339y_{1} + 0.3834x_{1} + -0.1158x_{3} + 0.6254x_{6}$$

In [230]:
fig = make_subplots(rows=2, cols=2)
fig.update_yaxes(title_text='Y2', row=1, col=1)
fig.update_yaxes(title_text='Y2', row=2, col=1)

fig.add_trace(px.scatter(x=y1, y=y2).data[0], row=1, col=1)
fig.update_xaxes(title_text='Y1', row=1, col=1)

fig.add_trace(px.scatter(x=x1, y=y2).data[0], row=1, col=2)
fig.update_xaxes(title_text='X1', row=1, col=2)

fig.add_trace(px.scatter(x=x3, y=y2).data[0], row=2, col=1)
fig.update_xaxes(title_text='X3', row=1, col=3)

fig.add_trace(px.scatter(x=x6, y=y2).data[0], row=2, col=2)
fig.update_xaxes(title_text='X6', row=2, col=1)

fig.update_layout(height=800, title='Графики с линиями тренда')
fig.show()

### Третье уравнение

$$y_3 = a_3 + a_{31}y_1 + a_{32}y_2 + b_{31}x_{11} + \epsilon$$

#### Исходная система уравнений

In [231]:
fn3_rec = pd.concat([y1, y2, x11], axis=1)
CustomLinearRegression(y3, fn3_rec).summary()

                                 OLS Regression Results                                
Dep. Variable:                     y3   R-squared (uncentered):                   0.997
Model:                            OLS   Adj. R-squared (uncentered):              0.996
Method:                 Least Squares   F-statistic:                              2142.
Date:                Mon, 24 Apr 2023   Prob (F-statistic):                    2.82e-27
Time:                        21:10:36   Log-Likelihood:                          52.186
No. Observations:                  25   AIC:                                     -98.37
Df Residuals:                      22   BIC:                                     -94.71
Df Model:                           3                                                  
Covariance Type:            nonrobust                                                  
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------

#### Исходная система уравнений после отбора признаков методом обратного исключения (Backward elimination)

In [237]:
CustomLinearRegression(y3, fn3_rec).backward_elimination(start=0)

                                 OLS Regression Results                                
Dep. Variable:                     y3   R-squared (uncentered):                   0.996
Model:                            OLS   Adj. R-squared (uncentered):              0.996
Method:                 Least Squares   F-statistic:                              5866.
Date:                Mon, 24 Apr 2023   Prob (F-statistic):                    3.38e-30
Time:                        21:11:26   Log-Likelihood:                          49.969
No. Observations:                  25   AIC:                                     -97.94
Df Residuals:                      24   BIC:                                     -96.72
Df Model:                           1                                                  
Covariance Type:            nonrobust                                                  
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------

#### Конечная система уравнений и веса

In [238]:
CustomLinearRegression(y3, pd.concat([y2], axis=1)).get_coefficients()

y2    0.988215
dtype: float64
$$\hat y_3 = 0.9882y_{2}$$


$$\hat y_3 = 0.9882y_{2}$$

In [240]:
fig = make_subplots(rows=1, cols=1)
fig.update_yaxes(title_text='Y3', row=1, col=1)

fig.add_trace(px.scatter(x=y2, y=y3).data[0], row=1, col=1)
fig.update_xaxes(title_text='Y1', row=1, col=1)

fig.update_layout(height=500, title='Графики с линиями тренда')
fig.show()

## Система одновременных уравнений

$$
\begin{cases}
y_1 = a_1 + b_{11}x_{1} + b_{12}x_{2} + \epsilon \\
y_2 = a_2 + a_{21}y_1 + b_{23}x_{3} + b_{24}x_{4} + b_{25}x_{5} + b_{26}x_{6} + b_{27}x_{7} + b_{28}x_{8} + b_{29}x_{9} + \epsilon \\
y_3 = a_3 + a_{31}y_1 + b_{310}x_{10} + b_{311}x_{11} + \epsilon \\
\end{cases}
$$

### Первое уравнение

$$y_1 = a_1 + b_{11}x_{1} + b_{12}x_{2} + \epsilon$$

#### Исходная система уравнений

In [241]:
fn1_odn = pd.concat([x1, x2], axis=1)
CustomLinearRegression(y1, fn1_odn, model_type=Poisson).summary()

Optimization terminated successfully.
         Current function value: 0.742222
         Iterations 6
                          Poisson Regression Results                          
Dep. Variable:                     y1   No. Observations:                   25
Model:                        Poisson   Df Residuals:                       23
Method:                           MLE   Df Model:                            1
Date:                Mon, 24 Apr 2023   Pseudo R-squ.:                 -0.2771
Time:                        21:14:24   Log-Likelihood:                -18.556
converged:                       True   LL-Null:                       -14.529
Covariance Type:            nonrobust   LLR p-value:                     1.000
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
x1            -2.2612      1.004     -2.253      0.024      -4.228      -0.294
x2             1.8198      1.

#### Исходная система уравнений после отбора признаков методом обратного исключения (Backward elimination)

In [247]:
CustomLinearRegression(y1, fn1_odn, model_type=Poisson).backward_elimination()

Optimization terminated successfully.
         Current function value: 0.742222
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.742222
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.742222
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.779372
         Iterations 6
                          Poisson Regression Results                          
Dep. Variable:                     y1   No. Observations:                   25
Model:                        Poisson   Df Residuals:                       24
Method:                           MLE   Df Model:                            0
Date:                Mon, 24 Apr 2023   Pseudo R-squ.:                 -0.3411
Time:                        21:15:14   Log-Likelihood:                -19.484
converged:                       True   LL-Null:                       -14.529
Covariance Type:            nonrobust  

#### Конечная система уравнений и веса

In [249]:
CustomLinearRegression(y1, pd.concat([x1], axis=1), model_type=Poisson).get_coefficients()

Optimization terminated successfully.
         Current function value: 0.779372
         Iterations 6
x1   -1.136666
dtype: float64
$$\hat y_1 = -1.1367x_{1}$$


$$\hat y_1 = -1.1367x_{1}$$

In [251]:
fig = make_subplots(rows=1, cols=1)
fig.update_yaxes(title_text='Y1', row=1, col=1)

fig.add_trace(px.scatter(x=x1, y=y1).data[0], row=1, col=1)
fig.update_xaxes(title_text='X1', row=1, col=1)

fig.update_layout(height=500, title='Графики с линиями тренда')
fig.show()

### Второе уравнение

$$y_2 = a_2 + a_{21}y_1 + b_{23}x_{3} + b_{24}x_{4} + b_{25}x_{5} + b_{26}x_{6} + b_{27}x_{7} + b_{28}x_{8} + b_{29}x_{9} + \epsilon \\$$

#### Исходная система уравнений

In [252]:
fn2_odn = pd.concat([y1, x3, x4, x5, x6, x7, x8, x9], axis=1)
CustomLinearRegression(y2, fn2_odn, model_type=Poisson).summary()

Optimization terminated successfully.
         Current function value: 0.730852
         Iterations 7
                          Poisson Regression Results                          
Dep. Variable:                     y2   No. Observations:                   25
Model:                        Poisson   Df Residuals:                       17
Method:                           MLE   Df Model:                            7
Date:                Mon, 24 Apr 2023   Pseudo R-squ.:                -0.03276
Time:                        21:16:29   Log-Likelihood:                -18.271
converged:                       True   LL-Null:                       -17.692
Covariance Type:            nonrobust   LLR p-value:                     1.000
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
y1            -4.4608      6.732     -0.663      0.508     -17.655       8.733
x3            -4.8831      4.

#### Исходная система уравнений после отбора признаков методом обратного исключения (Backward elimination)

In [257]:
CustomLinearRegression(y2, fn2_odn, model_type=Poisson).backward_elimination(start=1)

Optimization terminated successfully.
         Current function value: 0.730852
         Iterations 7
Optimization terminated successfully.
         Current function value: 0.730852
         Iterations 7
Optimization terminated successfully.
         Current function value: 0.730852
         Iterations 7
Optimization terminated successfully.
         Current function value: 0.730963
         Iterations 7
Optimization terminated successfully.
         Current function value: 0.733479
         Iterations 7
Optimization terminated successfully.
         Current function value: 0.746055
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.753484
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.763798
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.789408
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.789408
  

#### Конечная система уравнений и веса

In [259]:
CustomLinearRegression(y2, pd.concat([y1, x3], axis=1), model_type=Poisson).get_coefficients()

Optimization terminated successfully.
         Current function value: 0.789408
         Iterations 6
y1    2.247103
x3   -1.895398
dtype: float64
$$\hat y_2 = 2.2471y_{1} + -1.8954x_{3}$$


$$\hat y_2 = 2.2471y_{1} + -1.8954x_{3}$$

In [261]:
fig = make_subplots(rows=1, cols=2)
fig.update_yaxes(title_text='Y2', row=1, col=1)

fig.add_trace(px.scatter(x=y1, y=y2).data[0], row=1, col=1)
fig.update_xaxes(title_text='Y1', row=1, col=1)

fig.add_trace(px.scatter(x=x3, y=y2).data[0], row=1, col=2)
fig.update_xaxes(title_text='X3', row=1, col=2)

fig.update_layout(height=500, title='Графики с линиями тренда')
fig.show()

### Третье уравнение

$$y_3 = a_3 + a_{31}y_1 + b_{310}x_{10} + b_{311}x_{11} + \epsilon$$

#### Исходная система уравнений

In [262]:
fn3_odn = pd.concat([y1, x10, x11], axis=1)
CustomLinearRegression(y3, fn3_odn, model_type=Poisson).summary()

Optimization terminated successfully.
         Current function value: 0.757028
         Iterations 6
                          Poisson Regression Results                          
Dep. Variable:                     y3   No. Observations:                   25
Model:                        Poisson   Df Residuals:                       22
Method:                           MLE   Df Model:                            2
Date:                Mon, 24 Apr 2023   Pseudo R-squ.:                -0.07232
Time:                        21:19:09   Log-Likelihood:                -18.926
converged:                       True   LL-Null:                       -17.649
Covariance Type:            nonrobust   LLR p-value:                     1.000
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
y1           -14.0908      9.347     -1.508      0.132     -32.410       4.229
x10           19.4375     10.

#### Исходная система уравнений после отбора признаков методом обратного исключения (Backward elimination)

In [263]:
CustomLinearRegression(y3, fn3_odn, model_type=Poisson).backward_elimination()

Optimization terminated successfully.
         Current function value: 0.757028
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.757028
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.757028
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.803733
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.803733
         Iterations 6
                          Poisson Regression Results                          
Dep. Variable:                     y3   No. Observations:                   25
Model:                        Poisson   Df Residuals:                       23
Method:                           MLE   Df Model:                            1
Date:                Mon, 24 Apr 2023   Pseudo R-squ.:                 -0.1385
Time:                        21:19:13   Log-Likelihood:                -20.093
converged:      

#### Конечная система уравнений и веса

In [264]:
CustomLinearRegression(y3, pd.concat([x10, x11], axis=1)).get_coefficients()

x10    0.159175
x11    0.797510
dtype: float64
$$\hat y_3 = 0.1592x_{10} + 0.7975x_{11}$$


$$\hat y_3 = 0.1592x_{10} + 0.7975x_{11}$$

In [265]:
fig = make_subplots(rows=1, cols=2)
fig.update_yaxes(title_text='Y3', row=1, col=1)

fig.add_trace(px.scatter(x=x10, y=y3).data[0], row=1, col=1)
fig.update_xaxes(title_text='X11', row=1, col=1)

fig.add_trace(px.scatter(x=x11, y=y3).data[0], row=1, col=2)
fig.update_xaxes(title_text='X11', row=1, col=2)

fig.update_layout(height=500, title='Графики с линиями тренда')
fig.show()